In [ ]:
from IPython.core.display import display, HTML
from sklearn.model_selection import KFold,GroupKFold
from tqdm import tqdm
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import gc

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score
import lightgbm as lgb

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib

STACK=True

path_submissions = '/'

target_name = 'target'
scores_folds = {}

data_dir = '../input/optiver-realized-volatility-prediction/'

①多重共線性解除
②特徴削減
③外れ値有無
④SEED
⑤モデルパターン

これらを変更したモデルをそれぞれ訓練し、下記でテスト結果を算出

外れ値を抜いた予測値＊外れ値ではない確率＋外れ値×外れ値確率

In [ ]:
#とりあえずint32にしていい変数を返す
def reduce_64_int(df, verbose=True):
    li=[]
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    pass
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    pass
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    li.append(col)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    pass
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    pass
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    pass
                else:
                    pass
    return li

def reduce_64(df, verbose=True):
    li=[]
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    pass
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    pass
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    pass
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    pass
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    pass
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    li.append(col)
                else:
                    pass
    return li


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

def _roll(a, shift):
    """ Roll 1D array elements. Improves the performance of numpy.roll()"""
    if not isinstance(a, np.ndarray):
        a = np.asarray(a)
    idx = shift % len(a)
    return np.concatenate([a[-idx:], a[:-idx]])

def skewness(x):
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.skew(x)

def kurtosis(x):
    if not isinstance(x, pd.Series):
        x = pd.Series(x)
    return pd.Series.kurtosis(x)

def abs_energy(x):
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)

def minimum(x):
    return np.min(x)

def maximum(x):
    return np.max(x)

def max_over_min(series):
    if len(series)<2:
        return 0
    if np.min(series) == 0:
        return np.nan
    return np.max(series)/np.min(series)

def mean_abs_change(x):
    return np.mean(np.abs(np.diff(x)))

def number_peaks(x, n):
    """
    Calculates the number of peaks of at least support n in the time series x. A peak of support n is defined as a
    subsequence of x where a value occurs, which is bigger than its n neighbours to the left and to the right.
    """
    x_reduced = x[n:-n]

    res = None
    for i in range(1, n + 1):
        result_first = x_reduced > _roll(x, i)[n:-n]

        if res is None:
            res = result_first
        else:
            res &= result_first

        res &= x_reduced > _roll(x, -i)[n:-n]
    return np.sum(res)

number_peaks_2 = lambda x: number_peaks(x,2)
number_peaks_2.__name__ = 'number_peaks_2'

number_peaks_5 = lambda x: number_peaks(x,5)
number_peaks_5.__name__ = 'number_peaks_5'

number_peaks_10 = lambda x: number_peaks(x,10)
number_peaks_10.__name__ = 'number_peaks_10'

def count(series):
    return series.size

def absolute_sum_of_changes(x):
    return np.sum(np.abs(np.diff(x)))

def variance_std_ratio(x):
    y = np.var(x)
    if y != 0:
        return y/np.sqrt(y)
    else:
        return np.nan


# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

def log_return(series):
    return np.log(series).diff()

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def count_unique(series):
    return len(np.unique(series))

def read_train_test():
    train = pd.read_csv(data_dir+'train.csv')
    test = pd.read_csv(data_dir+'test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    return train,test


# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap


# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    df['bid_ask_ratio'] = (df['ask_size1']+df['ask_size2'])/(df['bid_size1']+df['bid_size2'])

    # Dict for aggregations
    create_feature_dict = {
        'wap1': [np.sum, np.std,skewness,kurtosis,abs_energy,minimum,maximum,max_over_min,number_peaks_5,mean_abs_change,absolute_sum_of_changes,variance_std_ratio],
        'wap2': [np.sum, np.std,skewness,kurtosis,abs_energy,minimum,maximum,max_over_min,number_peaks_5,mean_abs_change,absolute_sum_of_changes,variance_std_ratio],
        'wap3': [np.sum, np.std,skewness,kurtosis,abs_energy,minimum,maximum,max_over_min,number_peaks_5,mean_abs_change,absolute_sum_of_changes,variance_std_ratio],
        'wap4': [np.sum, np.std,skewness,kurtosis,abs_energy,minimum,maximum,max_over_min,number_peaks_5,mean_abs_change,absolute_sum_of_changes,variance_std_ratio],
        'log_return1': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'log_return2': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'log_return3': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'log_return4': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max,abs_energy,minimum,maximum,variance_std_ratio],
        'price_spread2':[np.sum, np.max,abs_energy,minimum,maximum,variance_std_ratio],
        'bid_spread':[np.sum, np.max,np.std,abs_energy,minimum,maximum,number_peaks_5,mean_abs_change],
        'ask_spread':[np.sum, np.max,np.std,abs_energy,minimum,maximum,number_peaks_5,mean_abs_change],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        'bid_ask_spread':[np.sum, np.max,np.std,abs_energy,minimum,maximum,number_peaks_5,mean_abs_change],
        'bid_ask_ratio':[np.sum, np.mean, np.std],
    }
    create_feature_dict_time = {
        'log_return1': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'log_return2': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'log_return3': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'log_return4': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
    }
    
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    create_feature_dict_time = {
        'log_return': [np.std,np.var,minimum,maximum,variance_std_ratio,realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    

    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        # vol vars
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
   
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200']


    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df


In [ ]:
#利用カラム設定
#USE_COLS=pd.read_pickle("../input/train-data-opt/train_f_remove.pickle").columns
USE_COLS=pd.read_pickle("../input/train-data-opt/train_f_marutico.pickle").columns
USE_COLS_TEST=[i for i in USE_COLS if "target"!=i]

# Read train and test
_train,test = read_train_test()
train =pd.read_pickle("../input/train-data-opt/train_f.pickle")

# Get unique stock ids 
#train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
#train_ = preprocessor(train_stock_ids, is_train = True)
#train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()

# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
#train = get_time_stock(train)
test = get_time_stock(test)

import gc
del _train,test_stock_ids
gc.collect()

#外れ値の代表
#MED=train[train["target"]>0.02]["target"].mean()

In [ ]:
# replace by order sum (tau)
train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )

train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )

# delta tau
train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']

colNames = [col for col in list(train.columns)
            if col not in {"stock_id", "time_id", "target", "row_id"}]
len(colNames)

In [ ]:
#float64を32に変換
features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
cols=reduce_64(train[features])
train[cols]=train[cols].astype("float32")
cols=reduce_64(test[features])
test[cols]=test[cols].astype("float32")

cols=reduce_64_int(train[features])
train[cols]=train[cols].astype("int32")
cols=reduce_64_int(test[features])
test[cols]=test[cols].astype("int32")

In [ ]:
from sklearn.cluster import KMeans
# making agg features

train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = train.loc[train['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()

mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_6c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_6c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_6c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_6c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_6c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_6c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_6c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1'] 

train = pd.merge(train,mat1[nnn],how='left',on='time_id')
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

import gc
del mat1,mat2,train_p,corr,ids
gc.collect()

In [ ]:
#float64を32に変換
features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
cols=reduce_64(train[features])
train[cols]=train[cols].astype("float32")
cols=reduce_64(test[features])
test[cols]=test[cols].astype("float32")

cols=reduce_64_int(train[features])
train[cols]=train[cols].astype("int32")
cols=reduce_64_int(test[features])
test[cols]=test[cols].astype("int32")

In [ ]:
"""
初期設定
"""
import time
from pandarallel import pandarallel
pandarallel.initialize()

path_to_files = "../input/optiver-realized-volatility-prediction"

book_train_files =  path_to_files + '/book_train.parquet/stock_id={}'
trade_train_files =  path_to_files + '/trade_train.parquet/stock_id={}'

book_test_files =  path_to_files + '/book_test.parquet/stock_id={}'
trade_test_files =  path_to_files + '/trade_test.parquet/stock_id={}'

SMALL_F = 0.00000001

np.random.seed(111)

cfg = dict(
    isCollectDataOnly = True,
    isStockIdUsed = False,
    isTFModelUsed = False,
    trainNotUsedCols = ['row_id', 'target', 'time_id', 'stock_id'],
    predictNotUsedCols = ['row_id', 'time_id', 'stock_id'],
    useHyperOpt = False,
    useLabelTransformation = False,
    volumeBarThreshold = 500.0
)

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

def log_return(prob):
    prob += SMALL_F
    return np.log(prob).diff()

def log_return2(x):
    return np.log1p(x.pct_change())

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))


def getOscStoch(x):
    return (x[-1] - np.min(x))/(np.max(x)-np.min(x)+SMALL_F)

def getNormVal(x):
    return (x[-1])/(np.mean(x)+SMALL_F)

def getBreath(ret, size):
    f = ret > 0
    upside = size[f]
    downside = size[~f]
    
    di = np.sum(upside)/(np.sum(downside)+SMALL_F)
    ado = np.sum(upside) - np.sum(downside)
    
    return di, ado
    
def rateLastFirst(x):
    return np.mean(x)/(np.sum(x)+SMALL_F)

def rolling_windows_vectorized(array, sub_window_size=2):
    start = 0
    max_time = len(array)-sub_window_size
    sub_windows = (
        start +
        # expand_dims are used to convert a 1D array to 2D array.
        np.expand_dims(np.arange(sub_window_size), 0) +
        np.expand_dims(np.arange(max_time + 1), 0).T
    )
    
    return array[sub_windows]

def rolling_windows_vectorized_v2(array, sub_window_size, stride_size):
    start = 0
    max_time = len(array)-sub_window_size
    sub_windows = (
        start + 
        np.expand_dims(np.arange(sub_window_size), 0) +
        # Create a rightmost vector as [0, V, 2V, ...].
        np.expand_dims(np.arange(max_time + 1, step=stride_size), 0).T
    )
    
    return array[sub_windows]    

def getVolumaBars(i_data, threshold=1000.0):
    o, h, l, c, v  = 0.0, 0.0, 1000000.0, 0.0, 0.0
    res_array = []
    isNewBar = True
    bar_index = 0.0
    cum_volume = 0.0
    data_len = i_data.shape[0]
    for i in range(data_len):
        #print(i_data[i])
        cur_price = i_data[i][0]
        #print(cur_price)
        if True == isNewBar:
            bar_index = i
            o = cur_price
            c = 0.0
            h = 0.0
            l = 10000000.0
            v = 0.0
            
            isNewBar = False

        if cur_price > h:
            h = cur_price
        if cur_price < l:
            l = cur_price

        v += i_data[i][1]
        
        if (v >= threshold) or (i == data_len-1):
            isNewBar = True
            #bar_index = i
            c = cur_price
            res_array.append([bar_index, o, h, l, c, v])


    return pd.DataFrame(res_array, columns=['bar_index', 'open', 'high', 'low', 'close', 'volume'])


def _get_beta(high, low, window):

    beta_r = np.empty(high.shape)
    beta_r[:] = np.NaN

    ret = np.log(high / low)
    high_low_ret = ret ** 2
    beta = rolling_windows_vectorized(high_low_ret, 2).sum(axis=1)
    beta = rolling_windows_vectorized(beta, window).mean(axis=1)
    beta_r[len(beta_r)-len(beta):] = beta    

    return beta_r


def _get_gamma(high, low):
    gamma_r = np.empty(high.shape)
    gamma_r[:] = np.NaN

    high_max = rolling_windows_vectorized(high, 2).max(axis=1)
    low_min = rolling_windows_vectorized(low, 2).min(axis=1)
    gamma = np.log(high_max / low_min) ** 2
    gamma_r[len(gamma_r)-len(gamma):] = gamma

    return gamma_r    

def get_bekker_parkinson_vol2(high, low, window: int = 20):

    beta = _get_beta(high, low, window)
    gamma = _get_gamma(high, low)

    k2 = (8 / np.pi) ** 0.5
    den = 3 - 2 * 2 ** .5
    sigma = (2 ** -0.5 - 1) * beta ** 0.5 / (k2 * den)
    sigma += (gamma / (k2 ** 2 * den)) ** 0.5
    sigma[sigma < 0] = 0

    return sigma

def get_garman_class_vol2(open, high, low, close, window):
    ret_value = np.empty(high.shape)
    ret_value[:] = np.NaN
    ret = np.log(high / low)  # High/Low return
    close_open_ret = np.log(close / open)  # Close/Open return
    estimator = 0.5 * ret ** 2 - (2 * np.log(2) - 1) * close_open_ret ** 2
    ret_v = rolling_windows_vectorized(estimator, window).mean(axis=1)
    ret_value[len(ret_value)-len(ret_v):] = np.sqrt(ret_v)
    return ret_value

def getWindows(n_bars, isSpecialWindow=False, min_bar_length=3):
    if(True==isSpecialWindow):
        window_size = n_bars//2
    else:
        window_size = n_bars-(min_bar_length-1)

    if(window_size<=0):
        if(n_bars-1 > 0):
            window_size = n_bars - 1
        else:
            window_size=1

    return window_size

def getMicrostructuralFeatures(input_df, output_df, col_prefix = '', col_postfix = '', min_bar_length = 3, volumeThreshold = cfg['volumeBarThreshold'], isParkinson=True, isGarman=True, isYyang=True, isBekker=True, isMicro=True, isSpecialWindow=False, micro_cols=[]):
    
    v_sum = np.sum(input_df.loc[:, 'size']).astype(np.float64)
    thres = volumeThreshold if v_sum >= volumeThreshold else v_sum
    volume_bars = getVolumaBars(input_df.loc[:,['price','size']].to_numpy(), thres)

    window_size = getWindows(len(volume_bars), isSpecialWindow=isSpecialWindow, min_bar_length=min_bar_length)

    if(True == isGarman):
        col_name = col_prefix+'garman_class_vol'+col_postfix
        pv = get_garman_class_vol2(volume_bars.open.to_numpy(), volume_bars.high.to_numpy(), volume_bars.low.to_numpy(), volume_bars.close.to_numpy(), window=window_size)
        pv = pv[~np.isnan(pv)]
        if(len(pv)>0):
            output_df.loc[:,col_name] = np.median(pv)
        else:
            output_df.loc[:,col_name] = 0.0

    if(True == isBekker):
        col_name = col_prefix+'bekker_parkinson_vol'+col_postfix
        pv = get_bekker_parkinson_vol2(volume_bars.high.to_numpy(), volume_bars.low.to_numpy(), window=window_size)
        pv = pv[~np.isnan(pv)]
        if(len(pv)>0):
            output_df.loc[:,col_name] = np.median(pv)
        else:
            output_df.loc[:,col_name] = 0.0

    return output_df

def getDataFromBidAsk_numpy(df, ci):
    a = 0
    b = 0
    spread  = {}
    for k in [1,2]:
        #k = i+1
        bidp = 'bid_price{}'.format(k)
        askp = 'ask_price{}'.format(k)
        bids = 'bid_size{}'.format(k)
        asks = 'ask_size{}'.format(k)
        #calculate comulative wap
        a += (df[:,ci[bidp]] * df[:,ci[asks]] + df[:,ci[askp]] * df[:,ci[bids]])
        b += df[:,ci[bids]] + df[:,ci[asks]]

        #wap 1 and 2
        spread[f'fb_w_{k}'] = (df[:,ci[bidp]] * df[:,ci[asks]] + df[:,ci[askp]] * df[:,ci[bids]] ) / (df[:,ci[bids]] + df[:,ci[asks]] + SMALL_F)
        spread[f'fb_mid_point_{k}'] = (df[:,ci[askp]]) + (df[:,ci[bidp]]) / 2
        spread[f'fb_volume_total_{k}'] = (df[:,ci[asks]]) + (df[:,ci[bids]])

    
    # mean wap
    spread['fb_w'] = (a/(b+SMALL_F))
    # rates
    spread['fb_w_rate'] = (spread['fb_w_1']) / (spread['fb_w_2']+SMALL_F) 
    spread['fb_mid_point_rate'] = (spread['fb_mid_point_1']) / (spread['fb_mid_point_2']+SMALL_F)
    #sum volume
    spread['fb_volume_total'] = spread['fb_volume_total_1'] + spread['fb_volume_total_2']

    
    ################# test ##################
    spread['ask_1'] = df[:,ci['ask_price1']]
    spread['bid_1'] = df[:,ci['bid_price1']]
    spread['ask_2'] = df[:,ci['ask_price2']]
    spread['bid_2'] = df[:,ci['bid_price2']]
    #########################################
    
    return spread



def Fx(group, stock_id=0, n=10):
    new_df = pd.DataFrame()
    name = int(group.time_id.unique()[0])
    tmp = pd.DataFrame()

    #calculate log return from the following features:
    cols = [
        'fb_w', 
        'fb_w_1', 
        'fb_w_2',
        'fb_mid_point_1',
        'fb_mid_point_rate',
        'fb_w_rate',
    ]

    new_cols = [s + '_lr' for s in cols]
    group.loc[:,new_cols] = log_return2(group[cols]).to_numpy()
    group = group[~group['fb_w'].isnull()]

    #calculate realized volatility
    cols = new_cols
    new_cols = [s + '_vola' for s in cols]
    tmp = pd.concat([tmp, pd.DataFrame(realized_volatility(group.loc[:,cols]).to_numpy().reshape(1,-1), columns=new_cols)], axis=1)

    #calculate sum of log return
    cols = [
        'fb_w_2_lr',
    ]
    new_cols = [s + '_sum' for s in cols]
    tmp = pd.concat([tmp, pd.DataFrame(np.sum(group.loc[:,cols]).to_numpy().reshape(1,-1), columns=new_cols)], axis=1)

    #calculate sqsum    
    cols = [
        'fb_w_lr', 
        'fb_w_1_lr', 
        'fb_w_2_lr',
        'fb_mid_point_1_lr',
    ]
    new_cols = [s + '_sqsum' for s in cols]
    tmp = pd.concat([tmp, pd.DataFrame(np.sum((group.loc[:,cols])**2).to_numpy().reshape(1,-1), columns=new_cols)], axis=1)
    
    #calculate book length
    tmp.loc[:,'book_length'] = [group.shape[0]]

    #calclulate market microstructural features
    c = '1'
    cols_1 = []
    new_df = pd.DataFrame({'price': group.loc[:,'fb_mid_point_'+c].to_numpy().flatten(), 'size': group.loc[:,'fb_volume_total_'+c].to_numpy().flatten()}).reset_index()
    tmp = getMicrostructuralFeatures(new_df, tmp, col_prefix = 'fb_', col_postfix = '_'+c, micro_cols=cols_1)
    c = '2'
    cols_2 = []  
    new_df = pd.DataFrame({'price': group.loc[:,'fb_mid_point_'+c].to_numpy().flatten(), 'size': group.loc[:,'fb_volume_total_'+c].to_numpy().flatten()}).reset_index()
    tmp = getMicrostructuralFeatures(new_df, tmp, col_prefix = 'fb_', col_postfix = '_'+c, micro_cols=cols_2)
    
    ############ test idea ################
    col_name = "Test_1"
    window_size = getWindows(len(group), isSpecialWindow=False, min_bar_length=3)
    pv = get_bekker_parkinson_vol2(group.ask_1.to_numpy(), group.bid_1.to_numpy(), window=window_size)
    pv = pv[~np.isnan(pv)]
    if(len(pv)>0):
        tmp.loc[:,col_name] = np.median(pv)
    else:
        tmp.loc[:,col_name] = 0.0

    col_name = "Test_2"
    window_size = getWindows(len(group), isSpecialWindow=False, min_bar_length=3)
    pv = get_bekker_parkinson_vol2(group.ask_2.to_numpy(), group.bid_2.to_numpy(), window=window_size)
    pv = pv[~np.isnan(pv)]
    if(len(pv)>0):
        tmp.loc[:,col_name] = np.median(pv)
    else:
        tmp.loc[:,col_name] = 0.0
    #######################################
    
    tmp.loc[:,'row_id'] = str(stock_id) + '-' + str(name)
    tmp.loc[:,'time_id'] = int(name)
    return tmp

def getFeaturesFromBookData(df, stock_id, n=10):
    results = df.groupby(['time_id']).parallel_apply(Fx, stock_id=stock_id, n=n).reset_index(drop=True)
    return results

def getDataFromTrade(df):
    log_ret = log_return(df.price).dropna()
    rz_vol = realized_volatility(log_ret)
    
    tmp = pd.DataFrame()

    
    tmp.loc[:,'p_vwap_my'] = [np.sum(df['price'].values*df['size'].values)/(np.sum(df['size'].values+SMALL_F))]
    tmp.loc[:,'p_rz_vol'] = rz_vol
    tmp.loc[:,'p_sqsum'] = np.sum(log_ret**2)
    tmp.loc[:,'p_sum'] = np.sum(log_ret)
    
    tmp.loc[:,'p_lr_rate'] = rateLastFirst(log_ret)
    
    tmp.loc[:,'p_price_count'] = count_unique(df['price'].to_numpy())
    tmp.loc[:,'p_sec_count'] = count_unique(df['seconds_in_bucket'].to_numpy())

    cols_p = []  
    tmp = getMicrostructuralFeatures(df.loc[:, ['price', 'size']], tmp, col_prefix = 'p_', col_postfix = '', isParkinson=False, isGarman=False, isYyang=False, micro_cols=cols_p)

    tmp.loc[:,'p_size_mean'] = np.mean(df['size']) 

    time_id = df.time_id.unique()[0]
    tmp.loc[:,'time_id'] = time_id
    return tmp

def getFeaturesFromTradeData(df):
    return df.groupby(['time_id']).parallel_apply(getDataFromTrade).reset_index(drop=True)

def constructPreprocessedDataFrame(file_path, isTrain):
    stock_id = file_path.split('=')[1]
    df_book_data = pd.read_parquet(file_path)
    if True == isTrain:
        df_trade_data =  pd.read_parquet(trade_train_files.format(stock_id))
    else:
        df_trade_data =  pd.read_parquet(trade_test_files.format(stock_id))

    print('Processing stock id:', stock_id)
    #display(df_book_data.time_id.unique())
    #preprocess book
    a = time.time()
    spread = getDataFromBidAsk_numpy(df_book_data.to_numpy(),{k: v for v, k in enumerate(df_book_data.columns.values)})
    df_book_data = pd.concat([df_book_data,pd.DataFrame(spread)], axis=1)
    df_book_datar = getFeaturesFromBookData(df_book_data, stock_id, 10)
    b = time.time()
    #print(f'preprocess book: {b-a}')
    
    #preprocess trade
    df_trade_datar = getFeaturesFromTradeData(df_trade_data)
    df_book_datar = df_book_datar.merge(df_trade_datar, on = ['time_id'], how = 'left')
    c = time.time()
    #print(f'preprocess trade: {c-b}')

    df_book_datar.loc[:,'stock_id'] = stock_id
    df_book_datar = df_book_datar.fillna(0.0)
    return df_book_datar

def constructBookDataDataFrame(list_file, isTrain=True):
    df_book = pd.DataFrame()
    for file in list_file:
        df_book = pd.concat([df_book, constructPreprocessedDataFrame(file, isTrain=isTrain)])
    return df_book

def getDataFromTransformedDataFx(df, prefix=''):
    cs = ['row_id', 'stock_id', 'time_id']
    used_cols = list(set(df.columns.to_list()) - set(cs))
    for c in used_cols:
        df.loc[:, prefix+c+'_rate'] = df.loc[:,c] / (np.mean(df.loc[:,c]) + SMALL_F)
        df.loc[:, prefix+c+'_diff'] = df.loc[:,c] - (np.mean(df.loc[:,c]) )

    return df

def getDataFromTransformedData(df):
    tmp1 = df.groupby(['stock_id']).parallel_apply(getDataFromTransformedDataFx, prefix='stock_id_').reset_index(drop=True)
    cols_tmp1 = [x for x in tmp1.columns if 'stock_id_' in x]
    cols_tmp1.append('stock_id')
    cols_tmp1.append('time_id')
    tmp2 = df.groupby(['time_id']).parallel_apply(getDataFromTransformedDataFx, prefix='time_id_').reset_index(drop=True)
    cols_tmp2 = [x for x in tmp2.columns if 'time_id_' in x]
    cols_tmp2.append('stock_id')
    cols_tmp2.append('time_id')

    df = df.merge(tmp1.loc[:,cols_tmp1], on = ['stock_id', 'time_id'], how = 'left')
    df = df.merge(tmp2.loc[:,cols_tmp2], on = ['stock_id', 'time_id'], how = 'left')
    
    print(df.shape)

    return df

In [ ]:
"""
実行
"""
#訓練データ
list_order_book_file_train = glob.glob(path_to_files + '/book_train.parquet/*')
#ret_df = constructBookDataDataFrame(list_order_book_file_train)
#ret_df = getDataFromTransformedData(ret_df)
ret_df = pd.read_pickle("../input/ref-df/ref_df.pkl")
cols=[i for i in ret_df.columns if i in USE_COLS] 
ret_df=ret_df[cols]

#テストデータ
list_order_book_file_train = glob.glob(path_to_files + '/book_test.parquet/*')
t_df = constructBookDataDataFrame(list_order_book_file_train, isTrain=False)
t_df = getDataFromTransformedData(t_df)
cols=[i for i in t_df.columns if i in USE_COLS] 
t_df=t_df[cols]

ret_df["stock_id"]=ret_df["stock_id"].astype(int)
ret_df["time_id"]=ret_df["time_id"].astype(int)

t_df["stock_id"]=t_df["stock_id"].astype(int)
t_df["time_id"]=t_df["time_id"].astype(int)

#容量削減
#float64を32に変換
features = [col for col in ret_df.columns if col not in {"time_id", "target", "row_id"}]
cols=reduce_64(ret_df[features])
ret_df[cols]=ret_df[cols].astype("float32")
t_df[cols]=t_df[cols].astype("float32")

"""
分散が小さい特徴量を削除する
from sklearn.feature_selection import VarianceThreshold

#正規化
_ret=ret_df.drop(["time_id","stock_id","row_id"],axis=1)
def minmax_norm(df):
    return (df - df.min()) / ( df.max() - df.min())
_ret = minmax_norm(_ret)

#分散で足きり
sel = VarianceThreshold(threshold=0.001)
sel.fit(_ret)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
drop_cols=_ret.loc[:, ~sel.get_support()].columns
ret_df=ret_df.drop(drop_cols,axis=1)
t_df=t_df.drop(drop_cols,axis=1)

import gc
del _ret
gc.collect()
"""

train = pd.merge(train,ret_df,how='left',on=['time_id','stock_id',"row_id"])
test = pd.merge(test,t_df,how='left',on=['time_id','stock_id',"row_id"])

import gc
del ret_df,t_df
gc.collect()

In [ ]:
#float64を32に変換
features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
cols=reduce_64(train[features])
train[cols]=train[cols].astype("float32")
cols=reduce_64(test[features])
test[cols]=test[cols].astype("float32")

cols=reduce_64_int(train[features])
train[cols]=train[cols].astype("int32")
cols=reduce_64_int(test[features])
test[cols]=test[cols].astype("int32")

In [ ]:
from sklearn.model_selection import KFold,GroupKFold

import lightgbm as lgb

def _prob(train,seed0):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': -1,
        'max_bin':100,
        'min_data_in_leaf':250,
        'learning_rate': 0.05,
        'subsample': 0.75,
        'subsample_freq': 4,
        'feature_fraction': 0.5,
        'lambda_l1': 0.5,
        'lambda_l2': 1.0,
        'n_jobs':-1,
        'seed':seed0,
        'feature_fraction_seed': seed0,
        'bagging_seed': seed0,
        'drop_seed': seed0,
        'data_random_seed': seed0,
    }
    train["class"]=train["target"].apply(lambda x:1 if x>0.02 else 0)
    train["weight"]=train["target"].apply(lambda x:10 if x>0.02 else 0.1)

    groups = train['time_id'].copy()

    features = [col for col in train.columns if col not in {"weight","time_id", "target", "row_id","class"}]
    y = train['class']

    oof_predictions = np.zeros(train.shape[0])
    test_prob = np.zeros(test.shape[0])
    kfold = GroupKFold(n_splits = 5)
    li= enumerate(kfold.split(train, y, groups))

    for fold, (trn_ind, val_ind) in li:
        print(f'Training fold {fold + 1}')
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        
        train_dataset = lgb.Dataset(x_train[features], y_train,weight=x_train["weight"])
        val_dataset = lgb.Dataset(x_val[features], y_val,weight=x_val["weight"])
        model = lgb.train(params = params,
                          num_boost_round=1000,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 200,
                          early_stopping_rounds=50,
                          )
        pred=model.predict(x_val[features])
        oof_predictions[val_ind] = pred
        test_prob += model.predict(test[features]) / 5
    """
    #データを別に保存
    from sklearn import metrics
    
    prob=train[["target","class"]]
    import gc
    del train
    gc.collect()

    prob["prob"]=oof_predictions
    prob[prob["class"]==1]["prob"]
    
    return prob,test_prob
    """
    return test_prob,oof_predictions

def train_and_evaluate_lgb(train, test, params):
    # Hyperparammeters (just basic)
    groups = train['time_id'].copy()

    features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
    y = train['target']
    # Create out of folds array
    oof_predictions = np.zeros(train.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(test.shape[0])
    # Create a KFold object
    #kfold = KFold(n_splits = 5, random_state = 11, shuffle = True)
    kfold = GroupKFold(n_splits = 5)
    li= enumerate(kfold.split(train, y, groups))

    # Iterate through each fold
    #for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
    for fold, (trn_ind, val_ind) in li:
        print(f'Training fold {fold + 1}')
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train[features], y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val, weight = val_weights)
        model = lgb.train(params = params,
                          num_boost_round=1000,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=50,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val[features])
        # Predict the test set
        test_predictions += model.predict(test[features]) / 5
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    lgb.plot_importance(model,max_num_features=20)
    # Return test predictions
    #return test_predictions
    return test_predictions,oof_predictions

In [ ]:
"""
外れ値(0.02以上かどうか)の確率を算出
2seed bagging
"""

test_prob1,oof_predictions1=_prob(train,0)
test_prob2,oof_predictions2=_prob(train,10)
test_prob=(test_prob1+test_prob2)/2
oof_predictions=(oof_predictions1+oof_predictions2)/2
import gc
del test_prob1,test_prob2
gc.collect()

"""
train["prob"]=oof_predictions
test["prob"]=test_prob

USE_COLS=list(USE_COLS)
USE_COLS_TEST=list(USE_COLS_TEST)
USE_COLS.append("prob")
USE_COLS_TEST.append("prob")
"""


In [ ]:
"""
外れ値なし、多重共線性解除データ
"""
seed0=210
params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':90,
    'min_data_in_leaf':500,
    'learning_rate': 0.06,
    'subsample': 0.7,
    'subsample_freq': 4,
    'feature_fraction': 0.5,
    'lambda_l1': 1.0,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}

# Traing and evaluate
predictions_lgb1,oof_predictions_lgb1= train_and_evaluate_lgb(
    train[train["target"]<=0.02][USE_COLS],test[USE_COLS_TEST],params0)

seed0=11
params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':100,
    'min_data_in_leaf':400,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.55,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
predictions_lgb2,oof_predictions_lgb2= train_and_evaluate_lgb(
    train[train["target"]<=0.02][USE_COLS],test[USE_COLS_TEST],params0)

seed0=111
params0 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':120,
    'min_data_in_leaf':300,
    'learning_rate': 0.04,
    'subsample': 0.75,
    'subsample_freq': 4,
    'feature_fraction': 0.6,
    'lambda_l1': 0.5,
    'lambda_l2': 0.5,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
predictions_lgb3,oof_predictions_lgb3= train_and_evaluate_lgb(
    train[train["target"]<=0.02][USE_COLS],test[USE_COLS_TEST],params0)

predictions_lgb1=(predictions_lgb1+predictions_lgb2+predictions_lgb3)/3
oof_predictions_lgb1=(oof_predictions_lgb1+oof_predictions_lgb2+oof_predictions_lgb3)/3

import gc
del predictions_lgb2,predictions_lgb3,oof_predictions_lgb2,oof_predictions_lgb3
gc.collect()

In [ ]:
"""
外れ値あり、多重共線性解除データ

大きい値を少し重み大きめ
"""
def train_and_evaluate_lgb(train, test, params):
    # Hyperparammeters (just basic)
    groups = train['time_id'].copy()

    features = [col for col in train.columns if col not in {"time_id", "target", "row_id"}]
    y = train['target']
    # Create out of folds array
    oof_predictions = np.zeros(train.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(test.shape[0])
    # Create a KFold object
    #kfold = KFold(n_splits = 5, random_state = 11, shuffle = True)
    kfold = GroupKFold(n_splits = 5)
    li= enumerate(kfold.split(train, y, groups))

    # Iterate through each fold
    #for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
    for fold, (trn_ind, val_ind) in li:
        print(f'Training fold {fold + 1}')
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        #"""
        train_weights = 1/np.abs(y_train)
        val_weights = 1/np.abs(y_val)
        train_dataset = lgb.Dataset(x_train[features], y_train, weight = train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val, weight = val_weights)
        """
        train_dataset = lgb.Dataset(x_train[features], y_train)
        val_dataset = lgb.Dataset(x_val[features], y_val)
        """
        model = lgb.train(params = params,
                          num_boost_round=1000,
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          verbose_eval = 250,
                          early_stopping_rounds=50,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val[features])
        # Predict the test set
        test_predictions += model.predict(test[features]) / 5
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    lgb.plot_importance(model,max_num_features=20)
    # Return test predictions
    #return test_predictions
    return test_predictions,oof_predictions

seed0=3
params1 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':90,
    'min_data_in_leaf':200,
    'learning_rate': 0.06,
    'subsample': 0.7,
    'subsample_freq': 4,
    'feature_fraction': 0.5,
    'lambda_l1': 1.0,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}

# Traing and evaluate
predictions_lgb21,oof_predictions_lgb21= train_and_evaluate_lgb(
    train[train["target"]<0.07][USE_COLS],test[USE_COLS_TEST],params1)

seed0=33
params1 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':100,
    'min_data_in_leaf':50,
    'learning_rate': 0.05,
    'subsample': 0.72,
    'subsample_freq': 4,
    'feature_fraction': 0.55,
    'lambda_l1': 0.5,
    'lambda_l2': 1.0,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
# Traing and evaluate
predictions_lgb22,oof_predictions_lgb22= train_and_evaluate_lgb(
    train[train["target"]<0.07][USE_COLS],test[USE_COLS_TEST],params1)


seed0=333
params1 = {
    'objective': 'rmse',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'max_bin':120,
    'min_data_in_leaf':25,
    'learning_rate': 0.04,
    'subsample': 0.75,
    'subsample_freq': 4,
    'feature_fraction': 0.6,
    'lambda_l1': 0.5,
    'lambda_l2': 0.5,
    'categorical_column':[0],
    'seed':seed0,
    'feature_fraction_seed': seed0,
    'bagging_seed': seed0,
    'drop_seed': seed0,
    'data_random_seed': seed0,
    'n_jobs':-1,
    'verbose': -1}
predictions_lgb23,oof_predictions_lgb23= train_and_evaluate_lgb(
    train[train["target"]<0.07][USE_COLS],test[USE_COLS_TEST],params1)

predictions_lgb3=(predictions_lgb21+predictions_lgb22+predictions_lgb23)/3
oof_predictions_lgb3=(oof_predictions_lgb21+oof_predictions_lgb22+oof_predictions_lgb23)/3

import gc
del predictions_lgb21,predictions_lgb22,predictions_lgb23,oof_predictions_lgb21,oof_predictions_lgb22,oof_predictions_lgb23
gc.collect()


In [ ]:
from numpy.random import seed
seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
import numpy as np
from keras import backend as K
def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

In [ ]:
# kfold based on the knn++ algorithm
out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

#out_train[out_train.isna().any(axis=1)]
out_train = out_train.fillna(out_train.mean())
out_train.head()

# code to add the just the read data after first execution

# data separation based on knn ++
nfolds = 5 # number of folds
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds) # number of individuals

# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]


lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)

lineNumber = np.sort(lineNumber)[::-1]

for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# saves index
for n in range(nfolds):
    
    values.append([lineNumber[n]])    


s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
         # saves the values of index           

        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []
train_nn=train[colNames].copy()
test_nn=test[colNames].copy()
#colNames.remove('row_id')
for col in colNames:
    #print(col)
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])    
    qt_train.append(qt)
train_nn[['stock_id','time_id','target']]=train[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']]=test[['stock_id','time_id']]

In [ ]:
#float64を32に変換
features = [col for col in train.columns if col not in {"time_id", "target", "row_id","class","weight"}]
cols=reduce_64(train[features])
train[cols]=train[cols].astype("float32")
cols=reduce_64(test[features])
test[cols]=test[cols].astype("float32")

cols=reduce_64_int(train[features])
train[cols]=train[cols].astype("int32")
cols=reduce_64_int(test[features])
test[cols]=test[cols].astype("int32")

In [ ]:
# making agg features
from sklearn.cluster import KMeans
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for ind in l:
    print(ind)
    newDf = train_nn.loc[train_nn['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test_nn.loc[test_nn['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()
mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])

nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',     
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',     
     'log_return1_realized_volatility_6c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_6c1',
     'trade_size_sum_0c1',
     'trade_size_sum_1c1', 
     'trade_size_sum_3c1',
     'trade_size_sum_4c1', 
     'trade_size_sum_6c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_6c1',      
     'price_spread_sum_0c1',
     'price_spread_sum_1c1',
     'price_spread_sum_3c1',
     'price_spread_sum_4c1',
     'price_spread_sum_6c1',   
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_6c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_6c1',   
     'volume_imbalance_sum_0c1',
     'volume_imbalance_sum_1c1',
     'volume_imbalance_sum_3c1',
     'volume_imbalance_sum_4c1',
     'volume_imbalance_sum_6c1',       
     'bid_ask_spread_sum_0c1',
     'bid_ask_spread_sum_1c1',
     'bid_ask_spread_sum_3c1',
     'bid_ask_spread_sum_4c1',
     'bid_ask_spread_sum_6c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_6c1'] 

#処理
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

train_nn = pd.merge(train_nn,mat1[nnn],how='left',on='time_id')
test_nn = pd.merge(test_nn,mat2[nnn],how='left',on='time_id')


In [ ]:
import gc
del train,test
del mat1,mat2,train_p,corr,ids,out_train
gc.collect()

In [ ]:
#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

#hidden_units = (128,64,32)
stock_embedding_size = 24

cat_data = train_nn['stock_id']

def base_model(hidden_units):
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(202,), name='num_data')#slightなし


    #embedding, flatenning and concatenating
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model


In [ ]:
"""
実行
"""
#訓練データ
list_order_book_file_train = glob.glob(path_to_files + '/book_train.parquet/*')
#ret_df = constructBookDataDataFrame(list_order_book_file_train)
#ret_df = getDataFromTransformedData(ret_df)
ret_df = pd.read_pickle("../input/ref-df/ref_df.pkl")
cols=[i for i in ret_df.columns if i in USE_COLS] 
ret_df=ret_df[cols]

#テストデータ
list_order_book_file_train = glob.glob(path_to_files + '/book_test.parquet/*')
t_df = constructBookDataDataFrame(list_order_book_file_train, isTrain=False)
t_df = getDataFromTransformedData(t_df)
cols=[i for i in t_df.columns if i in USE_COLS_TEST] 
t_df=t_df[cols]

ret_df["stock_id"]=ret_df["stock_id"].astype(int)
ret_df["time_id"]=ret_df["time_id"].astype(int)

t_df["stock_id"]=t_df["stock_id"].astype(int)
t_df["time_id"]=t_df["time_id"].astype(int)


#NNではいらない
ret_df=ret_df.drop("row_id",axis=1)
t_df=t_df.drop("row_id",axis=1)

#容量削減
#float64を32に変換
features = [col for col in ret_df.columns if col not in {"time_id", "target", "row_id"}]
cols=reduce_64(ret_df[features])
ret_df[cols]=ret_df[cols].astype("float32")
t_df[cols]=t_df[cols].astype("float32")


"""
分散が小さい特徴量を削除する
from sklearn.feature_selection import VarianceThreshold

#正規化
_ret=ret_df.drop(["time_id","stock_id"],axis=1)
def minmax_norm(df):
    return (df - df.min()) / ( df.max() - df.min())
_ret = minmax_norm(_ret)

#分散で足きり
sel = VarianceThreshold(threshold=0.001)
sel.fit(_ret)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
drop_cols=_ret.loc[:, ~sel.get_support()].columns
ret_df=ret_df.drop(drop_cols,axis=1)
t_df=t_df.drop(drop_cols,axis=1)

import gc
del _ret
gc.collect()
"""


#結合
train_nn = pd.merge(train_nn,ret_df,how='left',on=['time_id','stock_id'])
test_nn = pd.merge(test_nn,t_df,how='left',on=['time_id','stock_id'])


ret_df=ret_df.drop(['time_id','stock_id'],axis=1)
from tqdm import tqdm
for col in tqdm(ret_df.columns):
    #print(col)
    qt = QuantileTransformer(random_state=21,n_quantiles=2000, output_distribution='normal')
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])    
    qt_train.append(qt)

import gc
del ret_df,t_df
gc.collect()

In [ ]:
"""
特徴量の削減
"""
USE_COLS=[i for i in USE_COLS if i!="row_id"]
USE_COLS_TEST=[i for i in USE_COLS_TEST if i!="row_id"]

In [ ]:
#float64を32に変換
features = [col for col in train_nn.columns if col not in {"time_id", "target", "row_id","class","weight"}]
cols=reduce_64(train_nn[features])
train_nn[cols]=train_nn[cols].astype("float32")
cols=reduce_64(test_nn[features])
test_nn[cols]=test_nn[cols].astype("float32")

cols=reduce_64_int(train_nn[features])
train_nn[cols]=train_nn[cols].astype("int32")
cols=reduce_64_int(test_nn[features])
test_nn[cols]=test_nn[cols].astype("int32")

In [ ]:
def _pred(train_nn,test_nn,unit=(128,64,32),rate=0.006,se=0):
    from numpy.random import seed
    seed(se)
    import tensorflow as tf
    tf.random.set_seed(se)
    from tensorflow import keras

    target_name='target'
    scores_folds = {}
    model_name = 'NN'
    pred_name = 'pred_{}'.format(model_name)

    n_folds = 5
    groups = train_nn['time_id'].copy()
    kf = GroupKFold(n_splits = 5)
    y = train_nn['target']
    
    li= enumerate(kf.split(train_nn, y, groups))
    scores_folds[model_name] = []
    counter = 1

    features_to_consider = list(train_nn)

    features_to_consider.remove('time_id')
    features_to_consider.remove('target')
    try:
        features_to_consider.remove('pred_NN')
    except:
        pass

    train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
    test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
    train_nn[pred_name] = 0
    test_nn[target_name] = 0

    test_predictions_nn1 = np.zeros(test_nn.shape[0])
    oof_predictions_nn1 = np.zeros(train_nn.shape[0])

    #for n_count in range(n_folds):
    for n_count, (trn_ind, val_ind) in li:
        print('CV {}/{}'.format(counter, n_folds))
        print(train_nn)
        X_train = train_nn.loc[trn_ind, features_to_consider]
        y_train = train_nn.loc[trn_ind, target_name]
        X_test = train_nn.loc[val_ind, features_to_consider]
        y_test = train_nn.loc[val_ind, target_name]

        #############################################################################################
        # NN
        #############################################################################################

        model = base_model(unit)

        model.compile(
            keras.optimizers.Adam(learning_rate=rate),
            loss=root_mean_squared_per_error
        )

        try:
            features_to_consider.remove('stock_id')
        except:
            pass

        num_data = X_train[features_to_consider]

        scaler = MinMaxScaler(feature_range=(-1, 1))         
        num_data = scaler.fit_transform(num_data.values)    

        cat_data = X_train['stock_id']    
        target =  y_train

        num_data_test = X_test[features_to_consider]
        num_data_test = scaler.transform(num_data_test.values)
        cat_data_test = X_test['stock_id']

        model.fit([cat_data, num_data], 
                  target,               
                  batch_size=2048,
                  epochs=1000,
                  validation_data=([cat_data_test, num_data_test], y_test),
                  callbacks=[es, plateau],
                  validation_batch_size=len(y_test),
                  shuffle=True,
                 verbose = 1)

        preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
        oof_predictions_nn1[val_ind]=preds

        score = round(rmspe(y_true = y_test, y_pred = preds),5)
        print('Fold {} {}: {}'.format(counter, model_name, score))
        scores_folds[model_name].append(score)

        tt =scaler.transform(test_nn[features_to_consider].values)
        test_predictions_nn1 += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)/n_folds

        counter += 1
        features_to_consider.append('stock_id')
    #return test_predictions_nn1,oof_predictions_nn1
    return test_predictions_nn1

In [ ]:
"""
外れ値なし、特徴量削減
"""
test_predictions_nn1= _pred(
    train_nn[train_nn["target"]<=0.02].reset_index().drop("index",axis=1)[USE_COLS],
    test_nn[USE_COLS_TEST],unit=(128,64,32),rate=0.006,se=1)

test_predictions_nn2= _pred(
    train_nn[train_nn["target"]<=0.02].reset_index().drop("index",axis=1)[USE_COLS],
    test_nn[USE_COLS_TEST],unit=(256,128,64,32),rate=0.005,se=2)

test_prediction_nn1=(test_predictions_nn1+test_predictions_nn2)/2

import gc
del test_predictions_nn2
gc.collect()


In [ ]:
"""
全て、特徴量削減

2bagging
"""
test_predictions_nn21= _pred(
    train_nn[USE_COLS],
    test_nn[USE_COLS_TEST],unit=(128,64,32),rate=0.006,se=5)
test_predictions_nn22= _pred(
    train_nn[USE_COLS],
    test_nn[USE_COLS_TEST],unit=(256,128,64,32),rate=0.005,se=4)

test_prediction_nn3=(test_predictions_nn21+test_predictions_nn22)/2

import gc
del test_predictions_nn22, test_predictions_nn21
gc.collect()


In [ ]:
#"""
df=pd.DataFrame()
df["normal"]=(test_prediction_nn1+predictions_lgb1)/2#外れ値以外
df["all"]=(test_prediction_nn3+predictions_lgb3)/2#全て
df["prob"]=test_prob
df["target"]=0
df

idx=df[df["prob"]<0.5].index
df.loc[idx,"target"]=df.loc[idx,"normal"]
idx=df[df["prob"]>=0.5].index
df.loc[idx,"target"]=df.loc[idx,"all"]

test=pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test[target_name] = df["target"]
display(test[['row_id', target_name]].head(3))
test[['row_id', target_name]].to_csv('submission.csv',index = False)
#"""

In [ ]:
"""
df=pd.DataFrame()
df["normal"]=(test_prediction_nn1+predictions_lgb1)/2#外れ値以外
df["all"]=(test_prediction_nn3+predictions_lgb3)/2#全て
df["prob"]=test_prob
df["target"]=df["prob"]*df["all"]+(1-df["prob"])*df["normal"]


test=pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test[target_name] = df["target"]
display(test[['row_id', target_name]].head(3))
test[['row_id', target_name]].to_csv('submission.csv',index = False)
"""

In [ ]:
"""
df=pd.DataFrame()
df["target"]=(test_prediction_nn3+predictions_lgb3)/2#全て

test=pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
test[target_name] = df["target"]
display(test[['row_id', target_name]].head(3))
test[['row_id', target_name]].to_csv('submission.csv',index = False)
"""